In [2]:
%pip install opencv-python
%pip install Pillow

/var/folders/81/f9z4jcsj7gz1v7bs26nb55jm0000gn/T/ipykernel_85540/1087648453.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("transnetv2-pytorch-w

In [8]:
import torch
import ffmpeg
from transnet import TransNetV2
from inference import predict_video
import cv2
import numpy as np
import torch
import os
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import ttk

model = TransNetV2()
state_dict = torch.load("transnetv2-pytorch-weights.pth")
model.load_state_dict(state_dict)

<All keys matched successfully>

In [9]:
with torch.no_grad():
    # shape: batch dim x video frames x frame height x frame width x RGB (not BGR) channels
    scenes = predict_video('../../Test_dataset/00112.mp4', model)
    print("Detected scenes:", scenes)
    #visualize_detection('../../Dataset/00100.mp4', scenes,'./')

Trying for ../../Test_dataset/00112.mp4
[TransNetV2] Processing video frames 7676/7676Detected scenes: [[   0   71]
 [  73  134]
 [ 135  217]
 [ 218  302]
 [ 303  366]
 [ 367  460]
 [ 461  500]
 [ 501  533]
 [ 534  567]
 [ 568  616]
 [ 617  665]
 [ 666  698]
 [ 699  765]
 [ 766  795]
 [ 796  831]
 [ 832  865]
 [ 866  897]
 [ 898  934]
 [ 935  983]
 [ 984 1028]
 [1029 1126]
 [1127 1157]
 [1158 1195]
 [1196 1232]
 [1233 1263]
 [1264 1288]
 [1289 1468]
 [1469 1492]
 [1493 1528]
 [1529 1560]
 [1561 1630]
 [1631 1658]
 [1659 1694]
 [1695 1728]
 [1729 1820]
 [1821 1838]
 [1839 1853]
 [1854 1884]
 [1885 1926]
 [1927 1947]
 [1948 2004]
 [2005 2009]
 [2010 2019]
 [2020 2050]
 [2051 2091]
 [2092 2113]
 [2114 2154]
 [2155 2169]
 [2170 2195]
 [2196 2236]
 [2237 2277]
 [2278 2298]
 [2299 2414]
 [2415 2464]
 [2465 2500]
 [2501 2517]
 [2518 2546]
 [2547 2599]
 [2600 2641]
 [2642 2724]
 [2725 2763]
 [2764 2795]
 [2796 2831]
 [2832 2857]
 [2858 2877]
 [2878 2899]
 [2900 2938]
 [2939 2959]
 [2960 2982]


In [1]:


def extract_frame(video_path, frame_idx):
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
    ret, frame = cap.read()
    cap.release()
    if not ret:
        return None
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    return Image.fromarray(frame_rgb)

def visualize_shots_scrollable(video_path, scenes_data):
    if isinstance(scenes_data, torch.Tensor):
        scenes_data = scenes_data.cpu().numpy()
    elif isinstance(scenes_data, list):
        scenes_data = np.array(scenes_data)

    root = tk.Tk()
    root.title("Shot Boundary Viewer")
    def on_closing():
        root.quit()
        root.destroy()

    root.protocol("WM_DELETE_WINDOW", on_closing)

    main_frame = ttk.Frame(root)
    main_frame.pack(fill=tk.BOTH, expand=True)

    canvas = tk.Canvas(main_frame)
    scrollbar = ttk.Scrollbar(main_frame, orient=tk.VERTICAL, command=canvas.yview)
    scrollable_frame = ttk.Frame(canvas)

    scrollable_frame.bind(
        "<Configure>",
        lambda e: canvas.configure(
            scrollregion=canvas.bbox("all")
        )
    )

    canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
    canvas.configure(yscrollcommand=scrollbar.set)

    canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
    scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

    for idx, (start, end) in enumerate(scenes_data):
        start_img = extract_frame(video_path, int(start))
        end_img = extract_frame(video_path, int(end))

        if start_img is None or end_img is None:
            continue

        start_img_tk = ImageTk.PhotoImage(start_img.resize((160, 90)))
        end_img_tk = ImageTk.PhotoImage(end_img.resize((160, 90)))

        pair_frame = ttk.Frame(scrollable_frame, padding=10)
        pair_frame.pack(fill=tk.X)

        ttk.Label(pair_frame, text=f"Shot {idx+1}: [{start}-{end}]").pack(anchor="w")

        images_frame = ttk.Frame(pair_frame)
        images_frame.pack()

        ttk.Label(images_frame, image=start_img_tk).pack(side=tk.LEFT, padx=5)
        ttk.Label(images_frame, image=end_img_tk).pack(side=tk.LEFT, padx=5)

        # Store reference to avoid garbage collection
        pair_frame.start_img_tk = start_img_tk
        pair_frame.end_img_tk = end_img_tk

    root.mainloop()

if __name__ == '__main__':
    video_file = '../../Dataset/00001.mp4'
    #example_scenes = [[0, 433], [434, 495], [496, 553], [554, 646]]
    with torch.no_grad():
    # shape: batch dim x video frames x frame height x frame width x RGB (not BGR) channels
        scenes = predict_video(video_file, model)
    scenes_np = np.array(scenes)
    print(os.path.exists("00112.mp4"))
    visualize_shots_scrollable("00112.mp4", scenes_np)



In [5]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14", device=device)

image = preprocess(Image.open("cat.jpg")).unsqueeze(0).to(device)
image1 = preprocess(Image.open("car.png")).unsqueeze(0).to(device)
text = clip.tokenize(["a photo of a white and brown cat"]).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)

    print("Image features:", image_features.shape)
    print("Text features:", text_features.shape)
    im = torch.nn.functional.normalize(image_features[0], dim=0)
    print(im.min(), im.max(), im.mean())
    print("Diff: ", torch.nn.functional.cosine_similarity(im , torch.nn.functional.normalize(text_features, dim=0), dim=-1).item())
    
    logits_per_image, logits_per_text = model(torch.cat([image, image1], dim=0), text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()
    probs1 = logits_per_text.softmax(dim=-1).cpu().numpy()
    print("Image features:", probs1)

print("Top predictions:", probs)


Image features: torch.Size([1, 768])
Text features: torch.Size([1, 768])
tensor(-0.4855) tensor(0.5251) tensor(-0.0007)
Diff:  0.20456713438034058
Image features: [[1.0000000e+00 1.0508931e-08]]
Top predictions: [[1.]
 [1.]]


In [ ]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14", device=device)

image = preprocess(Image.open("cat.jpg")).unsqueeze(0).to(device)
text = clip.tokenize(["a white and brown cat"]).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)

    print("Image features:", image_features.shape)
    print("Text features:", text_features.shape)
    im_norm = torch.nn.functional.normalize(image_features, p=2, dim=1)
    txt_norm = torch.nn.functional.normalize(text_features, p=2,dim=1)
    print("Diff: ", torch.nn.functional.cosine_similarity(im_norm , txt_norm).item())
    
    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()
    probs1 = logits_per_text.softmax(dim=-1).cpu().numpy()

print("Image features:", probs1)
print("Top predictions:", probs)


Image features: torch.Size([1, 768])
Text features: torch.Size([1, 768])
tensor(-0.4855) tensor(0.5251) tensor(-0.0007)
Diff:  0.26497387886047363
Image features: [[1.]]
Top predictions: [[1.]]


In [31]:
import faiss
import numpy as np
import scipy
import torch
import faiss.contrib.torch_utils

# x_torch = torch.nn.functional.normalize(image_features, p=2, dim=1)
# q_torch = torch.nn.functional.normalize(text_features, p=2, dim=1)

x_torch = im_norm.clone()
q_torch = txt_norm.clone()


# If you need them back as numpy arrays for faiss
x = x_torch.numpy(force=True)
q = q_torch.numpy(force=True)

index = faiss.IndexFlatIP(x.shape[1])
index.add(x)
distance, idx = index.search(q, 1)
print('Distance by FAISS:{}'.format(distance))

# Cosine similarity for comparison
result = 1 - scipy.spatial.distance.cosine(x[0], q[0])
print('Cosine similarity:', result)

Distance by FAISS:[[0.26497385]]
Cosine similarity: 0.26497385695498765
